In [0]:
########################################Train ML Models#############################################


#import libraries
import os, sys
import re
import pickle
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.feature_extraction.text import *
from sklearn.model_selection import *
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.multiclass import *


#load xml contents of train and test datasets
xmlText = pickle.load(open('/gdrive/My Drive/Brainwaves/xmlTextClean.pk', 'rb'))
xmlTextTest = pickle.load(open('/gdrive/My Drive/Brainwaves/xmlTextTestClean.pk', 'rb'))


#load training targets
dfTarget = pd.read_csv('/gdrive/My Drive/Brainwaves/richFeatures.csv')

#load submission(test) file
psdf = pd.read_csv('/gdrive/My Drive/Brainwaves/public_submission.csv')

#test file name ids
testFileNames = psdf['File_Name'].values

x = [0]*len(dfTarget)

for i in range(len(dfTarget)):
    x[i]=xmlText[dfTarget.iloc[i, 1].lower()]
    
xTest = []

for fileName in testFileNames:
  xTest.append(xmlTextTest[fileName.lower()])

dfTarget=dfTarget.drop(labels=dfTarget.columns[0:2], axis=1)

x = np.array(x)

print(dfTarget.info())


#output fields
obtainColumns=[
       'TC_currency', 
       'TC_Bespoke',
       'calulation_agent', 
       'CA_dispute_resolution ', 
       'RD_method_party_A',
       'RD_party_A_moody_short_term_trigger_method',
       'RD_party_A_moody_short_term_threshold',
       'RD_party_A_SnP_short_term_trigger_method',
       'RD_party_A_SnP_short_term_threshold',
       'RD_party_A_Fitch_short_term_trigger_method',
       'RD_party_A_Fitch_short_term_threshold',
       'RD_party_A_moody_long_term_trigger_method',
       'RD_party_A_moody_long_term_threshold',
       'RD_party_A_SnP_long_term_trigger_method',
       'RD_party_A_SnP_long_term_threshold',
       'RD_party_A_Fitch_long_term_trigger_method',
       'RD_party_A_Fitch_long_term_threshold', 
       'governing law'
]

mlColumns = [
       'CA_fallback_default_dispute',
       'RD_party_A_short_term_debt_classification',
       'RD_party_A_long_term_debt_classification',
       'Cross_default_threshold_type',
       'Cross_default_measure',
       'governing law',
]

#prepare test prediction dataframe
dfPred = pd.DataFrame({'File_Name':testFileNames}, columns=[
       'File_Name', 
       'date',
       'party_A',
       'party B',
       'TC_currency',
       'TC_Bespoke',
       'calulation_agent', 'CA_fallback_default_dispute',
       'CA_dispute_resolution ', 'RD_method_party_A',
       'RD_party_A_short_term_debt_classification',
       'RD_party_A_moody_short_term_trigger_method',
       'RD_party_A_moody_short_term_threshold',
       'RD_party_A_SnP_short_term_trigger_method',
       'RD_party_A_SnP_short_term_threshold',
       'RD_party_A_Fitch_short_term_trigger_method',
       'RD_party_A_Fitch_short_term_threshold',
       'RD_party_A_long_term_debt_classification',
       'RD_party_A_moody_long_term_trigger_method',
       'RD_party_A_moody_long_term_threshold',
       'RD_party_A_SnP_long_term_trigger_method',
       'RD_party_A_SnP_long_term_threshold',
       'RD_party_A_Fitch_long_term_trigger_method',
       'RD_party_A_Fitch_long_term_threshold', 'Cross_default_threshold_type',
       'Cross_default_percentage/amount', 'Cross_default_measure',
       'multibranch_party_A', 'multibranch_party_B', 'governing law'
])

In [0]:
pkModels = {}

#Train a LGBMClassifier model on each output classifier field 
for column in obtainColumns+mlColumns:
  model = LGBMClassifier(n_jobs=-1, random_state=7)
  params = {
      'max_depth': [5, 7, 10, 15, 20],
      'num_leaves': [32, 50, 70, 100, 150],
      'num_estimators': [100, 150, 200],
      'learning_rate': [0.09, 0.1, 0.12],
      'subsample': [0.8, 0.9, 1.0],
      'subsample_freq': [0, 1, 3, 5],
      'colsample_bytree': [0.8, 0.9, 1.0],
  }
  
  #Randomized Search to find best params
  model = RandomizedSearchCV(model, params, n_iter=10, random_state=7, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=7), n_jobs=-1, verbose=1000)

  model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', model),
  ])
  model.fit(x, dfTarget.loc[:, column])
  
  pkModels[column] = model
  
  #predict output field for test data
  dfPred[column]=model.predict(xTest)
  
  print("DONE "+column)

In [0]:
#save models for quick access or for deploying through a server
import pickle
pickle.dump(pkModels, open('/gdrive/My Drive/Brainwaves/pkModels.pk', 'wb'))

In [0]:
#Fetch date of agreement using simple regex rules
xmlText2 = pickle.load(open('/gdrive/My Drive/Brainwaves/xmlText.pk', 'rb'))
xmlTextTest2 = pickle.load(open('/gdrive/My Drive/Brainwaves/xmlTextTest.pk', 'rb'))

xTest2 = []
for fileName in testFileNames:
  xTest2.append(xmlTextTest2[fileName.lower()])

import calendar

monthDic1={v: k for k,v in enumerate(calendar.month_abbr)}
monthDic = {}

for k in monthDic1:
  monthDic[k.lower()]=monthDic1[k]
  monthDic[k]=monthDic1[k]
  
print(monthDic)  

import re
count=0
gg=-1
for x in xTest2:
  gg+=1
  mm=re.finditer(r'[Dd]ated as.{30}', x)
  year='2000'
  month='00'
  day='00'
  for m in mm:
    ms = m.group(0)
    #regex for year
    yre=re.search(r'[0-9]{4}', ms) 
    if yre!=None:
      year=yre.group(0)
      
    #regex for day
    dre = re.search(r'[\s.^0-9][0-9]{1,2}[\s.rRtT,]', ms)
    if dre!=None:
      day=dre.group(0)[1:-1]
      if len(str(day))==1:
        day='0'+str(day)
        
    #logic for month extraction
    for k in monthDic:
      if k in ms:
        month=monthDic[k]
        if len(str(month))==1:
          month='0'+str(month)
     
  dfPred.loc[gg, 'date'] = str(year)+str(month)+str(day)
  print(dfPred.loc[gg, 'date'])



In [0]:
#Fetch party names using simple regex rules
kk=-1
for x in xTest2:
  kk+=1
  m=re.search(r'(.{1,100})and(.{1,100}) have entered', x)
  if m!=None:
    partyA=''
    partyB=''
    if 'socie' in m.group(1).lower():
      im=re.search(r'socie[^\s]*\s*[^\s,.]*', m.group(1).lower())
      partyA=im.group(0)
      partyB=m.group(2).replace('.', ' ').replace(',', ' ').strip()
      partyB=re.compile(r'[(),."\'“;-]').split(partyB)[0]
      partyB=re.compile(r'\s{2}').split(partyB)[0]
      
    if 'socie' in m.group(2).lower():
      im=re.search(r'socie[^\s]*\s*[^\s,.]*', m.group(2).lower())
      partyB=im.group(0)
      partyA = m.group(1)
      qq=re.search(r'((dated as of)|(\d){2})(.*)', partyA)
      if qq!=None:
        if qq.group(2)!=None:
          partyA = qq.group(2)
    
  else:
    print("Not found")
  if partyA.strip()=='':
    partyA='NotFound'
  if partyB.strip()=='':
    partyB='NotFound'
      
  dfPred.loc[kk, 'party_A']=partyA.strip()
  dfPred.loc[kk, 'party B']=partyB.strip()
  print(partyA+" ---- "+partyB)

In [0]:
#Obtain branches of the two parties
for i in range(len(dfPred)):
  aa=False
  bb=False
  if 'socie' in dfPred.loc[i, 'party_A']:
    aa=True
  elif 'socie' in dfPred.loc[i, 'party B']:
    bb=True
  if aa:
    
    dfPred.loc[i, 'multibranch_party_A']='Worldwide'
    
    dfPred.loc[i, 'multibranch_party_B']='No'
  if bb:
    dfPred.loc[i, 'multibranch_party_B']='Worldwide'
    dfPred.loc[i, 'multibranch_party_A']='No'  
      

In [0]:
#Prepare a submission file
dfPred.to_csv('submission'+'.csv', index=False)

#download file from colab into local
from google.colab import files
files.download('submission'+str(count2-1)+'.csv')